# WEEK 2 CODE

### Content:
* TASK 1 (a) - Multiresolution histogram for only one level.
* TASK 2 (a) - Evaluation of TASK 1 (a).
* TASK 1 (b) - Multiresolution histogram concatenating many levels.
* TASK 2 (b) - Evaluation of TASK 1 (b).
* TASK 3
* TASK 4
* TASK 5
* TASK 6

In [1]:
#IMPORTS
import sys
import cv2
import numpy as np
np.set_printoptions(threshold=np.inf)
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from os.path import isfile, join
from os import listdir
from sklearn import preprocessing
from scipy.spatial import distance
import ml_metrics as metrics
import pickle
from sklearn.metrics import recall_score,precision_score,f1_score

# TASK1 (a) - Multiresolution histogram for only one level.

### Histogrames de les imatges de "bbdd"

In [2]:
numberBins = 256
Level = 10 #Divisions per axis

In [3]:
imagesFolder = "./bbdd/"
histogram_bbdd_matrix = np.empty([0, numberBins*3*Level**2]) #Creem una matriu buida
#print(histogram_bbdd_matrix.shape)

def blockbasedHistogram_oneLevel(img,level):
    #Calculem tamany del block
    height, width = img.shape[:2]
    block_height = int(height/level)
    block_width = int(width/level)
    hist_img = np.empty([0,0])
    colorx = ('x','y','z')
    canal = 0
    for i,col in enumerate(colorx): #Bucle per recorrer els canals de cada imatge  
        for r in range(level): #Bucle per recorrer els blocs d'una mateixa row
            for c in range(level): #Bucle per recorrer els blocs d'una mateixa column
                
                block = img[r*block_height:r*block_height+block_height, c*block_width:c*block_width+block_width] 
                hist = cv2.calcHist([block],[i],None,[numberBins],[0,256]) #Calculem histogrames per blocs
                hist_t = hist.transpose()      
                #print(hist_t.shape)
                #print(hist_img.shape)
                if (i==0) and (r==0) and (c==0):
                    hist_img = hist_t
                else:
                    hist_img = np.concatenate((hist_img, hist_t), axis = 1)

    cv2.normalize(hist_img, hist_img, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    return hist_img
    
for filename in sorted(listdir(imagesFolder)): #Bucle per recorrer les imatges de la bbdd
    if(filename != '.DS_Store'):
        #print(imagesFolder + filename)
        print ("\r Processing image...   {}".format(imagesFolder+filename), end="")
        img = cv2.imread(imagesFolder + filename)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2Lab)
        hist_img = blockbasedHistogram_oneLevel(img,Level)
        histogram_bbdd_matrix = np.vstack((histogram_bbdd_matrix, hist_img))   


        

 Processing image...   ./bbdd/bbdd_00278.jpg

In [4]:
histogram_bbdd_matrix.shape

(279, 76800)

### Histogrames de les imatges de QSD2_W1

In [5]:
query2folder = './qsd2_w1/'
histogram_query2_matrix = np.empty([0, numberBins*3*Level**2])


def mask_generator(query_img):

    #************************************************************************************************    

    # MASK GENERATOR
    #************************************************************************************************
    
    gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # noise removal
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

    # sure background area
    sure_bg = cv2.dilate(opening,kernel,iterations=3) #Fals. El sure 

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,3)
    ret, sure_fg = cv2.threshold(dist_transform,0.15*dist_transform.max(),255,0) #0.15 26.57% 

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)

    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)

    markers[markers != 0] = 200

    #35
    markers[0:77,:] = 100
    sure_fg[0:77,:] = 100
    markers[len(markers)-78:len(markers)-1,:] = 100
    sure_fg[len(markers)-78:len(markers)-1,:] = 100
    markers[:,0:77] = 100
    sure_fg[:,0:77] = 100
    markers[:,len(markers[0,:])-78:len(markers[0,:])-1] = 100
    sure_fg[:,len(markers[0,:])-78:len(markers[0,:])-1] = 100

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1

    # Now, mark the region of unknown with zero
    markers[sure_fg == 0] = 0

    markersres = cv2.watershed(query_img,markers)

    markersres[markersres == 201] = 255
    markersres[markersres == 101] = 0
    markersres[markersres == -1] = 255

    #Make the mask binary
    markersres = markersres/np.max(markersres)

    #print(markersres.dtype)
    mask = markersres.astype('uint8') 

    #opening (dilation + erosion) per eliminar pixels que son 1 a les cantonades de la mascara
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel)
    
    return mask
    
def blockbasedHistogram_oneLevel_maskRemoval(query_img, mask, level):
    
    #Compute the histogram
    query_img = cv2.cvtColor(query_img,cv2.COLOR_BGR2Lab)
    
    #************************************************************************************************    

    # CROP IMAGE
    #************************************************************************************************

    # Mask of non-black pixels (assuming image has a single channel).
    mask_rows, mask_columns = mask.shape
    mask_crop = mask > 0

    # Coordinates of non-black pixels.
    coords = np.argwhere(mask_crop)

    # Bounding box of non-black pixels.
    x0, y0 = coords.min(axis=0)
    x1, y1 = coords.max(axis=0) + 1   # slices are exclusive at the top

    # Get the contents of the bounding box.
    cropped = query_img[x0:x1, y0:y1]
    mask_cropped = mask[x0:x1, y0:y1]

   
    #************************************************************************************************    


    #Compute block size
    height, width = cropped.shape[:2]
    block_height = int(height/level)
    block_width = int(width/level)
    hist_img = np.empty([0,0])
    
    color = ('b','g','r')
    
    for i,col in enumerate(color):
        #Loop over blocks for each row
        for r in range(level):
            #Loop over blocks for each column
            for c in range(level):
                #print("canal: ", i)
                #print("r    : ", r)
                #print("c    : ", c)
                block = cropped[r*block_height:r*block_height+block_height, c*block_width:c*block_width+block_width] 
                mask_block = mask_cropped[r*block_height:r*block_height+block_height, c*block_width:c*block_width+block_width]
                hist = cv2.calcHist([block],[i],mask_block,[numberBins],[0,256]) #Calculem histogrames per blocs
                hist_t = hist.transpose()  
                #print(hist_t.shape)
                #print(hist_img.shape)
                if (i==0) and (r==0) and (c==0):
                    hist_query = hist_t
                    #print(hist_query.shape)
                else:
                    hist_query = np.concatenate((hist_query,hist_t), axis = 1)
                    #print(hist_query.shape)

    #print(histogram_query2_matrix.shape)
    #print(hist_query.shape)
    cv2.normalize(hist_query, hist_query, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    return hist_query


for filename in sorted(listdir(query2folder)):
    if(filename != '.DS_Store' and (filename.split('.')[1] == 'jpg')):
        
        print ("\r Processing image...   {}".format(query2folder+filename), end="")
        query_img = cv2.imread(query2folder + filename)
        
        # Mask genetation
        mask = mask_generator(query_img)
        
        #Save to the histogram_query matrix
        hist_query = blockbasedHistogram_oneLevel_maskRemoval(query_img, mask, Level)
        histogram_query2_matrix = np.vstack((histogram_query2_matrix, hist_query))

 Processing image...   ./qsd2_w1/00029.jpg

In [6]:
print(histogram_query2_matrix.shape)

(30, 76800)


# TASK 2 (a) - Evaluation of TASK 1 (a).

### Definition of the functions used to evaluate Task1

In [7]:
def computeMatrixRetrieval(histogram_bbdd, histogram_query, K=10):
    dst = np.zeros((30, 279))
    matrix_retrieval = np.zeros((30,K))

    for query_image_index in range(0,len(histogram_query)): 
        for bbdd_image_index in range(0, len(histogram_bbdd)):

            dst[query_image_index,bbdd_image_index] = distance.braycurtis(histogram_query[query_image_index,:], histogram_bbdd[bbdd_image_index,:])

        matrix_retrieval[query_image_index,:] = np.argsort(dst[query_image_index,:])[:K]
        
    #Convertir idx a list of lists
    matrix_retrieval = matrix_retrieval.astype(int)
    matrix_retrieval_lst = matrix_retrieval.tolist()
    return matrix_retrieval_lst

def evaluationTask1(matrix_retrieval, K=10):  
    #LLEGIR GT
    with open(query2folder + 'gt_corresps.pkl', 'rb') as fd:
            ll = pickle.load(fd)
            #print(ll)
    gt = np.empty((0,0))

    ll_prp = np.zeros((len(ll),1))

    for i in range(0,len(ll)):
        ll_prp[i] = ll[i][0]

    ll_prp_lst = ll_prp.tolist()

    mapak = metrics.average_precision.mapk(ll_prp_lst, matrix_retrieval, k=K)    
    score = mapak*100
    #print("SCORE: ",score, "%")

    #GUARDAR PICKLE RESULT
    pickle_out = open("result_qsd2.pkl", "wb")
    pickle.dump(matrix_retrieval, pickle_out)
    pickle_out.close()
    
    return score

### Evaluation

In [8]:
K=10
matrix_retrieval = computeMatrixRetrieval(histogram_bbdd_matrix, histogram_query2_matrix, K)
evaluationTask1(matrix_retrieval, K)

81.66666666666667

# TASK 1 (b) - Multiresolution histogram concatenating many levels.

### Histogrames de les imatges de "bbdd"

In [9]:
numberBins = 256
LevelMax = 5 #Divisions per axis

In [10]:
imagesFolder = "./bbdd/"

numeroDeBlocs = 0; #Variable per saber quants histogrames de blocs es calcularan i aixi inicialitzar el tamany de la variable histogram_bbdd_matrix 
for l in range(LevelMax):
    numeroDeBlocs = (l+1)**2 + numeroDeBlocs
#print("numeroDeBlocs:", numeroDeBlocs)
    
histogram_bbdd_matrix_blocs = np.empty([0, numberBins*3*numeroDeBlocs]) #Creem una matriu buida
#print("histogram_bbdd_matrix.shape", histogram_bbdd_matrix.shape)

def blockbasedHistogram_concatenatingLevels(img,levelMax):
    
    for lvl in range(levelMax): #Bucli per cobrir tots els levels i concatenarlos

        #Calculem tamany del block
        level = lvl + 1
        #print("level: ", level)
        height, width = img.shape[:2]
        block_height = int(height/level)
        block_width = int(width/level)
        colorx = ('x','y','z')
        canal = 0

        for i,col in enumerate(colorx): #Bucle per recorrer els canals de cada imatge  
            for r in range(level): #Bucle per recorrer els blocs d'una mateixa row
                for c in range(level): #Bucle per recorrer els blocs d'una mateixa column

                    block = img[r*block_height:r*block_height+block_height, c*block_width:c*block_width+block_width] 
                    hist = cv2.calcHist([block],[i],None,[numberBins],[0,256]) #Calculem histogrames per blocs
                    hist_t = hist.transpose()      
                    #print(hist_t.shape)
                    if (i==0) and (r==0) and (c==0) and (lvl==0):
                        hist_img = hist_t
                    else:
                        hist_img = np.concatenate((hist_img, hist_t), axis = 1)

                    #print(hist_img.shape)


    cv2.normalize(hist_img, hist_img, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    return hist_img

for filename in sorted(listdir(imagesFolder)): #Bucle per recorrer les imatges de la bbdd
    if(filename != '.DS_Store'):
        print ("\r Processing image...   {}".format(imagesFolder+filename), end="")
        img = cv2.imread(imagesFolder + filename)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2Lab)
        hist_img = np.empty([0,0])
        hist_img = blockbasedHistogram_concatenatingLevels(img, LevelMax)
        histogram_bbdd_matrix_blocs = np.vstack((histogram_bbdd_matrix_blocs, hist_img))   

 Processing image...   ./bbdd/bbdd_00278.jpg

In [11]:
histogram_bbdd_matrix_blocs.shape

(279, 42240)

### Histogrames de les imatges de QSD2_W1

In [12]:
query2folder = './qsd2_w1/'
histogram_query2_matrix = np.empty([0, numberBins*3*numeroDeBlocs])

def blockbasedHistogram_concatenatingLevels_maskRemoval(query_img, mask, levelMax):
    
    #Compute the histogram
    query_img = cv2.cvtColor(query_img,cv2.COLOR_BGR2Lab)
    
    #************************************************************************************************    

    # CROP IMAGE
    #************************************************************************************************
    
    # Mask of non-black pixels (assuming image has a single channel).
    mask_rows, mask_columns = mask.shape
    mask_crop = mask > 0

    # Coordinates of non-black pixels.
    coords = np.argwhere(mask_crop)

    # Bounding box of non-black pixels.
    x0, y0 = coords.min(axis=0)
    x1, y1 = coords.max(axis=0) + 1   # slices are exclusive at the top

    # Get the contents of the bounding box.
    cropped = query_img[x0:x1, y0:y1]
    mask_cropped = mask[x0:x1, y0:y1]
    
    #************************************************************************************************    

    # CALCULEM ELS HISTOGRAMES DE LES IMATGES RETALLADES
    #************************************************************************************************
    hist_img = np.empty([0,0])
    
    color = ('b','g','r')

    for lvl in range(levelMax): #Bucli per cobrir tots els levels i concatenarlos

        level = lvl + 1
        #Calculem tamany del block
        height, width = cropped.shape[:2]
        block_height = int(height/level)
        block_width = int(width/level)

        for i,col in enumerate(color):
            for r in range(level): #Bucle per recorrer els blocs d'una mateixa row
                for c in range(level): #Bucle per recorrer els blocs d'una mateixa column
                    #print("canal: ", i)
                    #print("r    : ", r)
                    #print("c    : ", c)
                    block = cropped[r*block_height:r*block_height+block_height, c*block_width:c*block_width+block_width] 
                    mask_block = mask_cropped[r*block_height:r*block_height+block_height, c*block_width:c*block_width+block_width]
                    hist = cv2.calcHist([block],[i],mask_block,[numberBins],[0,256]) #Calculem histogrames per blocs
                    hist_t = hist.transpose()  
                    #print(hist_t.shape)
                    #print(hist_img.shape)
                    if (i==0) and (r==0) and (c==0) and (lvl==0):
                        hist_query = hist_t
                        #print(hist_query.shape)
                    else:
                        hist_query = np.concatenate((hist_query,hist_t), axis = 1)
                        #print(hist_query.shape)

    #print(histogram_query2_matrix.shape)
    #print(hist_query.shape)
    cv2.normalize(hist_query, hist_query, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    return hist_query
    
for filename in sorted(listdir(query2folder)):
    if(filename != '.DS_Store' and (filename.split('.')[1] == 'jpg')):
        print ("\r Processing image...   {}".format(query2folder+filename), end="")
        query_img = cv2.imread(query2folder + filename)
        
        # Mask genetation
        mask = mask_generator(query_img)
        
        #Save to the histogram_query matrix   
        hist_query = blockbasedHistogram_concatenatingLevels_maskRemoval(query_img, mask, LevelMax)   
        histogram_query2_matrix = np.vstack((histogram_query2_matrix, hist_query))

 Processing image...   ./qsd2_w1/00029.jpg

In [13]:
print(histogram_query2_matrix.shape)

(30, 42240)


# TASK 2 (b) - Evaluation of TASK 1 (b).

In [14]:
K=3
matrix_retrieval = computeMatrixRetrieval(histogram_bbdd_matrix_blocs, histogram_query2_matrix, K)
evaluationTask1(matrix_retrieval, K)

61.11111111111111

### Histogrames de les imatges de QSD2_W1

# TASK 3 & TASK 4 & TASK 5

### Definition of the functions used for following Tasks 3, 4, 5 and 6

In [15]:
from sklearn.metrics import recall_score, precision_score, f1_score
from PIL import ImageDraw, Image

def detect_corners(mask):
    """
    Finds four points corresponding to rectangle corners

    :param mask: (ndarray) binary image
    :return: (int) points from corners
    """

    width = mask.shape[1]
    height = mask.shape[0]
    coords = np.argwhere(np.ones([height, width]))
    coords_x = coords[:, 1]
    coords_y = coords[:, 0]

    coords_x_filtered = np.extract(mask, coords_x)
    coords_y_filtered = np.extract(mask, coords_y)
    max_br = np.argmax(coords_x_filtered + coords_y_filtered)
    max_tr = np.argmax(coords_x_filtered - coords_y_filtered)
    max_tl = np.argmax(-coords_x_filtered - coords_y_filtered)
    max_bl = np.argmax(-coords_x_filtered + coords_y_filtered)

    tl_x, tl_y = int(coords_x_filtered[max_tl]), int(coords_y_filtered[max_tl])
    tr_x, tr_y = int(coords_x_filtered[max_tr]), int(coords_y_filtered[max_tr])
    bl_x, bl_y = int(coords_x_filtered[max_bl]), int(coords_y_filtered[max_bl])
    br_x, br_y = int(coords_x_filtered[max_br]), int(coords_y_filtered[max_br])

    return tl_x, tl_y, bl_x, bl_y, br_x, br_y, tr_x, tr_y

def text_detect_method1(img, opt = 0):
    """
    Text bounding box detection

    :param img: (ndarray) query image
    :param opt: (int) options to corner detection (0 or 1)
    :return: bbox: (tuple of int) bounding box, (tlx, tly, brx, bry)
    """

    bifilter = cv2.bilateralFilter(img, 9, 300, 300)

    hsv = cv2.cvtColor(bifilter, cv2.COLOR_BGR2HSV)

    ret, thresh = cv2.threshold(hsv[:, :, 1], 0, 255, cv2.THRESH_BINARY_INV)

    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, np.ones((7, 7), np.uint8))
    open1 = cv2.morphologyEx(closing, cv2.MORPH_OPEN, np.ones((10, 1), np.uint8), iterations=2)
    open2 = cv2.morphologyEx(open1, cv2.MORPH_OPEN, np.ones((1, 10), np.uint8), iterations=2)

    # plt.figure()
    # plt.imshow(open2, cmap = 'gray')

    ret, labels = cv2.connectedComponents(open2)
    # plt.figure()
    # plt.imshow(labels, cmap = 'gray')

    area = []
    for i, lab in enumerate(np.unique(labels)):
        area.append(open2[labels == lab].size)
    idx = sorted(range(len(area)), key=lambda k: area[k])

    x_n = open2.shape[0]
    nbb = list(range(int((x_n / 2) - x_n * (0.05)), int((x_n / 2) + x_n * (0.05))))

    if ret > 2:
        for i, j in enumerate(idx):
            if np.sum(open2[labels == j] == 0) == 0:
                idn = np.where(labels == j)[0]
                nocenter = [val for val in idn.tolist() if val in nbb]

                if (len(area) - i) > 2:
                    open2[labels == j] = 0

                if (len(area) - i) <= 2 and len(nocenter) > 0:
                    open2[labels == j] = 0
                    open2[labels == idx[i - 1]] = 255



    if opt == 0:

        # Coordinates of non-black pixels.
        coords = np.argwhere(open2)

        # Bounding box of non-black pixels.
        x0, y0 = coords.min(axis=0)
        x1, y1 = coords.max(axis=0) + 1

    else:
        y0, x0, _, _, y1, x1, _, _ = detect_corners(open2)

    bbox = (y0, x0, y1, x1)

    return bbox

def evaluationTask5(matrix_retrieval, K=10):  
    #LLEGIR GT
    with open('./qsd1_w2/' + 'gt_corresps.pkl', 'rb') as fd:
            ll = pickle.load(fd)
            #print(ll)
    gt = np.empty((0,0))

    ll_prp = np.zeros((len(ll),1))

    for i in range(0,len(ll)):
        ll_prp[i] = ll[i][0]

    ll_prp_lst = ll_prp.tolist()

    mapak = metrics.average_precision.mapk(ll_prp_lst, matrix_retrieval, k=K)    
    score = mapak*100
    #print("SCORE: ",score, "%")

    #GUARDAR PICKLE RESULT
    pickle_out = open("./result_qsd1_w2.pkl", "wb")
    pickle.dump(matrix_retrieval, pickle_out)
    pickle_out.close()
    
    return score

### Text detection and evaluation using bounding box

In [17]:
from evaluation.bbox_iou import bbox_iou

query2folder = './qsd1_w2/'

with open(query2folder+'text_boxes.pkl', 'rb') as fd:
    ll = pickle.load(fd)


IoU = []
histogram_query1w2_matrix = np.empty([0, numberBins*3*Level**2])

bboxes1 = []

for filename in sorted(listdir(query2folder)):
    count = 0
    if (filename != '.DS_Store' and (filename.split('.')[1] == 'jpg')):

        # Read image and its mask
        query_img = cv2.imread(query2folder + filename)
        mask = np.ones((query_img.shape[0],query_img.shape[1]),dtype='uint8')*255
        
        # Image number
        img_num = int(filename[0:5])
        bbox_gt = ll[img_num][0]

        # Search bbox for text
        bbox = text_detect_method1(query_img)

        # Set tl and br for predicted and ground truth
        y0, x0, y1, x1 = bbox
        bboxes1.append([[y0, x0, y1, x1]])
        y0_gt, x0_gt, y1_gt, x1_gt = bbox_gt
        
        # Save image with both bbox
        #cv2.rectangle(query_img, (y0, x0), (y1, x1), (0, 0, 255), 2)
        #cv2.rectangle(query_img, (y0_gt, x0_gt), (y1_gt, x1_gt), (0, 255, 0), 2)
        #cv2.imwrite('bbox_'+filename, query_img)

        ### TASK 4 ###
    
        # Compute metric IoU
        IoU.append(bbox_iou(bbox, bbox_gt))
        
        ### TASK 5 ###
        
        # Ignore pixels from bbox
        mask[x0:x1, y0:y1] = 0
        
        print ("\r Processing image...   {}".format(query2folder+filename), end="")
        
        #Save to the histogram_query matrix
        hist_query = blockbasedHistogram_oneLevel_maskRemoval(query_img, mask, Level)
        histogram_query1w2_matrix = np.vstack((histogram_query1w2_matrix, hist_query))

#GUARDAR PICKLE RESULT
pickle_out = open("boxes1.pkl", "wb")
pickle.dump(bboxes1, pickle_out)
pickle_out.close()
#print('\n\nMetric mean IoU:', np.mean(IoU))

 Processing image...   ./qsd1_w2/00029.jpg

# TASK 5 (b) - Evaluation

In [18]:
K=10
matrix_retrieval = computeMatrixRetrieval(histogram_bbdd_matrix, histogram_query1w2_matrix, K)
evaluationTask5(matrix_retrieval, K)

87.22222222222221

# TASK 6

### Definition of the functions used for Tasks 6

In [19]:
from PIL import ImageDraw, Image
from sklearn.metrics import recall_score, precision_score, f1_score

def waterShed(query_img, sat, part):
    """
    Applies watershed to detect foreground and baskground

    :param query_img: (ndarray) query image
    :param sat: (ndarray) query image saturation
    :param part: (int) one paint(0), two paints left part(1),  two paints right part(2)
    :return: mask: predicted mask
    """

    ret, thresh = cv2.threshold(sat, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    thresh = 255 - thresh

    # noise removal
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
    # fillin holes
    kernel = np.ones((5, 5), np.uint8)
    opening = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel, iterations=2)

    # sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations=3)  # Fals. El sure

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 3)
    ret, sure_fg = cv2.threshold(dist_transform, 0.13 * dist_transform.max(), 255, 0)  # 0.15 26.57%

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)

    markers[markers != 0] = 200

    border_top = int(sat.shape[0] * .03)

    if part != 2:
        border_r = int(sat.shape[1] * .01)
        border_l = int(sat.shape[1] * .02)
    if part != 1:
        border_r = int(sat.shape[1] * .02)
        border_l = int(sat.shape[1] * .01)


    # define known sure background
    markers[0:border_top, :] = 100
    sure_fg[0:border_top, :] = 100

    markers[len(markers) - border_top:len(markers) - 1, :] = 100
    sure_fg[len(markers) - border_top:len(markers) - 1, :] = 100

    markers[:, 0:border_l] = 100
    sure_fg[:, 0:border_l] = 100
    markers[:, len(markers[0, :]) - border_r:len(markers[0, :]) - 1] = 100
    sure_fg[:, len(markers[0, :]) - border_r:len(markers[0, :]) - 1] = 100

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1

    # Now, mark the region of unknown with zero
    markers[sure_fg == 0] = 0

    markersres = cv2.watershed(query_img, markers)

    markersres[markersres == 201] = 255
    markersres[markersres == 101] = 0
    markersres[markersres == -1] = 255

    # Make the mask binary
    return markersres / np.max(markersres)


def crop(query_im, mask_crop):
    # Coordinates of non-black pixels.
    coords = np.argwhere(mask_crop)

    # Bounding box of non-black pixels.
    x0, y0 = coords.min(axis=0)
    x1, y1 = coords.max(axis=0) + 1   # slices are exclusive at the top

    # Get the contents of the bounding box.
    cropped = query_im[x0:x1, y0:y1]
    mask_cropped = mask_crop[x0:x1, y0:y1]
    
    return cropped, mask_cropped, x0, y0
    


def computeMatrixRetrievalTask6(histogram_bbdd, histogram_query, img_list, K=10):
    
    dst = np.zeros((histogram_query.shape[0], 279))
    matrix_retrieval = np.zeros((histogram_query.shape[0],K))
    
    for query_image_index in range(0,len(histogram_query)): 
        for bbdd_image_index in range(0, len(histogram_bbdd)):

            dst[query_image_index,bbdd_image_index] = distance.braycurtis(histogram_query[query_image_index,:], histogram_bbdd[bbdd_image_index,:])

        matrix_retrieval[query_image_index,:] = np.argsort(dst[query_image_index,:])[:K]
    
    new_matrix_retrieval = []
    index_per_matrix_retrieval = 0
    for ind, num_images in enumerate(img_list):
        if(len(num_images)>1):
            aux = []
            aux_1 = (matrix_retrieval[index_per_matrix_retrieval].astype(int)).tolist()
            aux_2 = (matrix_retrieval[index_per_matrix_retrieval+1].astype(int)).tolist()
            aux.append(aux_1)
            aux.append(aux_2)
            new_matrix_retrieval.append(aux)
            index_per_matrix_retrieval = index_per_matrix_retrieval + 2
        else:
            aux = []
            aux_1 = (matrix_retrieval[index_per_matrix_retrieval].astype(int)).tolist()
            aux_2 = []
            aux.append(aux_1)
            aux.append(aux_2)
            new_matrix_retrieval.append(aux)
            index_per_matrix_retrieval = index_per_matrix_retrieval + 1
    
    return new_matrix_retrieval


def evaluationTask6(matrix_retrieval, K=10):
    #LLEGIR GT
    with open('./qsd2_w2/' + 'gt_corresps.pkl', 'rb') as fd:
            ll = pickle.load(fd)
            
    for l in range(len(ll)):
        if len(ll[l]) < 2:
            ll[l].append([])
            
    mapak = metrics.average_precision.mapk(ll, matrix_retrieval, k=K)    
    score = mapak*100
    print("SCORE: ",score, "%")

    #GUARDAR PICKLE RESULT
    pickle_out = open("result_qsd2_w2.pkl", "wb")
    pickle.dump(matrix_retrieval, pickle_out)
    pickle_out.close()
    
    return score


### Detect paintings, remove background and text and return correspondences

In [20]:
# initialize pixel mask
mask_gt_pixel = np.empty([1])
mask_pred_pixel = np.empty([1])

query2folder = './qsd2_w2/'
histogram_query2w2_matrix = np.empty([0, numberBins*3*Level**2])

img_l = []
bboxes = []

for filename in sorted(listdir(query2folder)):
    count = 0
    if (filename != '.DS_Store' and (filename.split('.')[1] == 'jpg')):

        # Read query image
        mask_gt = cv2.cvtColor(cv2.imread(query2folder + filename[0:-3] + 'png'), cv2.COLOR_BGR2GRAY) / 255
        query_img = cv2.imread(query2folder + filename)
        gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)
        
        # Get saturation from query image
        sat = cv2.cvtColor(query_img, cv2.COLOR_BGR2HSV)[:,:,1]
        
        # Divide image in two if weight is 1.6 superior than height
        if sat.shape[1] >= sat.shape[0]*1.6:
            
            # Predict masks for each half part of the image
            markersres1 = waterShed(query_img[:, 0:int(sat.shape[1]/2)],sat[:, 0:int(sat.shape[1]/2)], 1)
            markersres2 = waterShed(query_img[:, int(sat.shape[1] / 2)-1:-1], sat[:, int(sat.shape[1] / 2)-1:-1], 2)
            
            # Concatenate predicitons
            markersres = np.concatenate((markersres1, markersres2), axis=1)
            
            # Remove imperfections with morphological filers (fill holes and remove noise)
            markersres = cv2.morphologyEx(markersres, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)
            markersres = cv2.morphologyEx(markersres, cv2.MORPH_CLOSE, np.ones((1,40), np.uint8), iterations=1)
            markersres = cv2.morphologyEx(markersres, cv2.MORPH_CLOSE, np.ones((40, 1), np.uint8), iterations=1)

        else :
            # Predict mask
            markersres = waterShed(query_img, sat, 0)
            
            # Remove imperfections with morphological filers (noise)
            markersres = cv2.morphologyEx(markersres, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8), iterations=1)

        # Set predicted mask
        mask_pred = markersres.astype('uint8')

        # Get connected components
        ret, labels = cv2.connectedComponents(mask_pred)
        
        # Initialize mask for paintings
        mask = np.zeros((mask_pred.shape[0], mask_pred.shape[1], 2), dtype="uint8")
        
        # If exist more than two connected components (background and one painting)
        if ret > 2:

            # Compute area of each connected component
            area = []
            for i, lab in enumerate(np.unique(labels)):
                area.append(mask_pred[labels == lab].size)
                
            # Sort indexes from length of areas
            idx = sorted(range(len(area)), key=lambda k: area[k])

            # Check if more than three connected components (background and two paintings)
            if ret > 3:
                for i, j in enumerate(idx):
                    # Remove smallest areas
                    if (len(area)-i) > 3:
                        mask_pred[labels == j] = 0
            
            # Remove indexes labels where its pixels are now zero
            idx = [n for _, n in enumerate(idx) if np.sum(mask_pred[labels == n]) != 0]
            
            # Positions of non-zero elements of each painting
            obj1 = np.where(labels == idx[0])
            obj2 = np.where(labels == idx[1])
            
            # Determine if there exist two paintings
            if (area[idx[0]] > .28*area[idx[1]]):
                
                # Check that one connected component is not inside the other
                if (((np.min(obj1[1][:]) - np.max(obj2[1][:])) < 0) and ((np.max(obj1[1][:]) - np.min(obj2[1][:])) > 0)) \
                        or (((np.min(obj2[1][:]) - np.max(obj1[1][:])) < 0) and
                            ((np.max(obj2[1][:]) - np.min(obj1[1][:])) > 0)):
                    mask = mask_pred
                else:
                    # Set one mask to each painting
                    mask[labels == idx[0], 0] = 1
                    mask[labels == idx[1], 1] = 1

            else:
                # Set mask for the painting
                mask = mask_pred

        else:
            # Set mask for the painting
            mask = mask_pred

        ## POLYGON - Generate a polygon from mask corners ##
        
        mask_f = np.zeros((mask_pred.shape[0], mask_pred.shape[1], 2), dtype="uint8")
        
        # For two paintings
        if len(mask.shape) > 2:
            # Get Corners
            corners0 = detect_corners(mask[:, :, 0])
            
            # Draw polygon
            poly0 = Image.new('RGB', (mask_pred.shape[1], mask_pred.shape[0]), (0))
            pdraw0 = ImageDraw.Draw(poly0)
            pdraw0.polygon(xy=corners0, fill=(1), outline=None)
            mask_pred_f = np.array(poly0)[:, :, 0]
            mask_f[:, :, 0] = mask_pred_f
            
            # Get Corners
            corners1 = detect_corners(mask[:, :, 1])
            
            # Draw polygon
            poly1 = Image.new('RGB', (mask_pred.shape[1], mask_pred.shape[0]), (0))
            pdraw1 = ImageDraw.Draw(poly1)
            pdraw1.polygon(xy=corners1, fill=(1), outline=None)
            mask_f[:, :, 1] = np.array(poly1)[:, :, 0]
            mask_pred_f = mask_pred_f + mask_f[:, :, 1]

        # Just one painting
        else:
            # Get Corners
            corners = detect_corners(mask)
            
            # Draw polygon
            poly = Image.new('RGB', (mask_pred.shape[1], mask_pred.shape[0]), (0))
            pdraw = ImageDraw.Draw(poly)
            pdraw.polygon(xy=corners, fill=(1), outline=None)
            mask_pred_f = np.array(poly)[:, :, 0]
            mask_f = mask_pred_f
            
        # Save mask as image
        #cv2.imwrite(filename[0:-3]+'png', mask_pred_f*255)

        # Concatenate all pixels from all images
        #mask_gt_pixel = np.concatenate((mask_gt_pixel, mask_gt.flatten()), axis=0)
        #mask_pred_pixel = np.concatenate((mask_pred_pixel, mask_pred_f.flatten()), axis=0)
        
        print ("\r Processing image...   {}".format(query2folder+filename), end="")
        
        if len(mask.shape) > 2:
            
            img_l.append([0,0])
            
            box = []
            
            for p in range(2):
                
                cropped, mask_cropped, X0, Y0 = crop(query_img,mask_f[:,:,p])
                
                # Detect bbox
                bbox = text_detect_method1(cropped)
                y0, x0, y1, x1 = bbox
                mask_cropped[x0:x1, y0:y1] = 0
                box.append([Y0+y0, X0+x0, Y0+y1, X0+x1])
                
                #cv2.rectangle(query_img, (Y0+y0, X0+x0), (Y0+y1, X0+x1), (0, 0, 255), 2)
                #cv2.imwrite('bbox_'+filename, query_img)
                
                #Save to the histogram_query matrix each painting histogram
                hist_query = blockbasedHistogram_oneLevel_maskRemoval(cropped, mask_cropped, Level)
                histogram_query2w2_matrix = np.vstack((histogram_query2w2_matrix, hist_query))
                
            bboxes.append(box)
            
        else:
            
            img_l.append([0])
            
            cropped, mask_cropped, X0, Y0 = crop(query_img,mask_f)

            bbox = text_detect_method1(cropped)
            y0, x0, y1, x1 = bbox
            mask_cropped[x0:x1, y0:y1] = 0
            
            #cv2.rectangle(query_img, (Y0+y0, X0+x0), (Y0+y1, X0+x1), (0, 0, 255), 2)
            #cv2.imwrite('bbox_'+filename, query_img)
            
            bboxes.append([[Y0+y0, X0+x0, Y0+y1, X0+x1]])

            #Save to the histogram_query matrix each painting histogram
            hist_query = blockbasedHistogram_oneLevel_maskRemoval(cropped, mask_cropped, Level)
            histogram_query2w2_matrix = np.vstack((histogram_query2w2_matrix, hist_query))
            

# Mask evaluation
#mask_gt_pixel = mask_gt_pixel[1:]
#mask_pred_pixel = mask_pred_pixel[1:]

#PRECISION
#precision = precision_score(mask_gt_pixel, mask_pred_pixel)
#RECALL
#recall = recall_score(mask_gt_pixel, mask_pred_pixel)
#F1
#f1 = f1_score(mask_gt_pixel, mask_pred_pixel)

#print('PRECISION: ', precision, 'RECALL: ', recall, ' F1-SCORE: ', f1)

#GUARDAR PICKLE RESULT
pickle_out = open("text_boxes2.pkl", "wb")
pickle.dump(bboxes, pickle_out)
pickle_out.close()


 Processing image...   ./qsd2_w2/00029.jpg

In [21]:
matrix_retrieval = computeMatrixRetrievalTask6(histogram_bbdd_matrix, histogram_query2w2_matrix, img_l, K=10)
evaluationTask6(matrix_retrieval, K=10)

SCORE:  19.166666666666668 %


19.166666666666668